In [1]:
import os, sys
import subprocess
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
from pandas.tools.plotting import table
import seaborn as sns
from IPython.display import clear_output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

/usr/local/lib/python2.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
val=pd.read_excel('/data/Gurpreet/Echo/View_point2.xlsx', sheetname='Test', header=0)
for i in xrange(0,(val.size/4)):
    patnumber=val.iloc[i,0]
    Videonumber=val.iloc[i,1]
    Totalframes=val.iloc[i,2]
    score1=np.zeros((1,7))
    score2=np.zeros((1,7))
    score3=np.zeros((1,7))
    clear_output()
    for x in xrange(0,(Totalframes-1)):
        tf.reset_default_graph()
        x=x+1
        impath_beg="/data/Gurpreet/View_Class_OCR/Views4/Testing_Images/"
        #image_path="/data/Gurpreet/Echo/Test_Images/EQo_"+str(patnumber)+"_"+str(Videonumber)+"_"+str(x)+".jpg"
        #image_path="/data/Gurpreet/VC/AVIFiles/Evideo_"+str(patnumber)+"_"+str(x)+".jpg"
        image_path=str(impath_beg)+str(patnumber)+"_"+str(Videonumber)+"_"+str(x)+".jpg"
        image_data = tf.gfile.FastGFile(image_path, 'rb').read()
        label_lines = [line.rstrip() for line 
                   in tf.gfile.GFile("/data/Gurpreet/View_Class_OCR/Views4/retrained_labels4.txt")]
        # Unpersists graph from file
        with tf.gfile.FastGFile("/data/Gurpreet/View_Class_OCR/Views4/retrained_graph4.pb", 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            tf.import_graph_def(graph_def, name='')
        with tf.Session() as sess:
        # Feed the image_data as input to the graph and get first prediction
                softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
                predictions = sess.run(softmax_tensor, {'DecodeJpeg/contents:0': image_data})
        predictions_max=(predictions == predictions.max(axis=1, keepdims=True)).astype(int)
        score1=np.add(score1,predictions).reshape(1,7)
        predictions_normalized = normalize(predictions, norm='max')
        score2=np.add(score2,predictions_normalized).reshape(1,7)
        score3=np.add(score3,predictions_max.astype(int)).reshape(1,7)
        perc1=np.multiply(np.divide(score1,np.sum(score1)),100).reshape(1,7)
        perc2=np.multiply(np.divide(score2,np.sum(score2)),100).reshape(1,7)
        perc3=np.multiply(np.divide(score3,np.sum(score3)),100).reshape(1,7)
        labels=label_lines
        label_lines=np.array(label_lines).reshape(1,7)
        print ('Sr.no %d/%d Patient number %d Video number %d Total_frames %d completed %d' %(i+1,(val.size/3),patnumber,Videonumber,Totalframes,x))
       
    Result=np.concatenate((label_lines,score1,perc1,score2,perc2,score3,perc3),axis=0)
    Result2=pd.DataFrame(data=Result[1:,0:],columns=Result[0,0:],index=["PS","PS%","PSM","PSM%","MS","MS%"])
    Result2=Result2.convert_objects(convert_numeric=True).round(2)
    #Result2=Result2.round(3)
    print Result2
    sizes = Result2.iloc[5,:]
    colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue','red','green']
    explode = (0.1, 0, 0, 0)  # explode 1st slice
    plt.pie(sizes,labels=labels, colors=colors)
    piefname=str(impath_beg)+"Results/PIE/PIE_EQo_"+str(patnumber)+"_"+str(Videonumber)+"_"+str(x)+".jpg"
    plt.savefig(piefname)
    plt.close()
    Result2.plot.bar()
    barfname=str(impath_beg)+"Results/BAR/BAR_EQo_"+str(patnumber)+"_"+str(Videonumber)+"_"+str(x)+".jpg"
    plt.savefig(barfname)
    plt.close()
    sns.heatmap(Result2,annot=True, fmt="2.2f")
    heatfname=str(impath_beg)+"Results/HEAT/HEAT_EQo_"+str(patnumber)+"_"+str(Videonumber)+"_"+str(x)+".jpg"
    plt.savefig(heatfname)
    plt.close()

Sr.no 158/210 Patient number 102 Video number 51 Total_frames 33 completed 1
Sr.no 158/210 Patient number 102 Video number 51 Total_frames 33 completed 2
Sr.no 158/210 Patient number 102 Video number 51 Total_frames 33 completed 3
Sr.no 158/210 Patient number 102 Video number 51 Total_frames 33 completed 4
Sr.no 158/210 Patient number 102 Video number 51 Total_frames 33 completed 5
Sr.no 158/210 Patient number 102 Video number 51 Total_frames 33 completed 6
Sr.no 158/210 Patient number 102 Video number 51 Total_frames 33 completed 7
Sr.no 158/210 Patient number 102 Video number 51 Total_frames 33 completed 8
Sr.no 158/210 Patient number 102 Video number 51 Total_frames 33 completed 9
Sr.no 158/210 Patient number 102 Video number 51 Total_frames 33 completed 10
Sr.no 158/210 Patient number 102 Video number 51 Total_frames 33 completed 11
Sr.no 158/210 Patient number 102 Video number 51 Total_frames 33 completed 12
Sr.no 158/210 Patient number 102 Video number 51 Total_frames 33 complete